In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Patent Search with RAG-based Reasoning Engine, AlloyDB and LangChain



## Overview

[LangChain on Vertex AI](https://cloud.google.com/vertex-ai/generative-ai/docs/reasoning-engine/overview)
is a managed service that helps you to build and deploy LangChain apps to a managed Reasoning Engine runtime.

RAG (Retrieval-Augmented Generation) is an AI framework that combines the strengths of traditional information retrieval systems (such as databases) with the capabilities of generative large language models (LLMs).  By combining this extra knowledge with its own language skills, the AI can write text that is more accurate, up-to-date, and relevant to your specific needs.

## Objectives

In this tutorial, you will learn how to build and deploy an agent (model, tools, and reasoning) using the Vertex AI SDK for Python and AlloyDB for PostgreSQL LangChain integration.

Your [LangChain](https://python.langchain.com/docs/get_started/introduction) agent will use an [AlloyDB Vector Store](https://github.com/googleapis/langchain-google-alloydb-pg-python/tree/main) to perform a similary search and retrieve related data to ground the LLM response.

* Install and set up the AlloyDB for PostgreSQL for LangChain and the Vertex AI Python SDKs
* Create an AlloyDB cluster and instance
* Create an AlloyDB database user
* Define a retriever to perform similarity searches
* Use the LangChain agent template provided in the Vertex AI SDK for Reasoning Engine
* Deploy and test your agent on Reasoning Engine in Vertex AI


## Use Case
Patent Search based on contextual relevance

## Assumption
1. **You must have completed the steps 2, 3, 4 and 5 in this codelab: first**
https://codelabs.developers.google.com/patent-search-alloydb-geminilab

2. You have this notebook implemented in the same account as your Google Cloud project account.

### Install and import dependencies

In [ ]:
!pip install --upgrade --quiet "google-cloud-aiplatform[reasoningengine,langchain]" langchain-google-alloydb-pg langchain-google-vertexai

In [ ]:
!pip install langchain-google-genai

In [ ]:
!pip install google-cloud-aiplatform[reasoningengine,langchain]==1.57.0 langchain-google-alloydb-pg==0.4.1 langchain-google-vertexai==1.0.4

In [3]:
from typing import List
import uuid

from langchain_core.documents import Document
from langchain_google_alloydb_pg import AlloyDBEngine, AlloyDBVectorStore
from langchain_google_vertexai import VertexAIEmbeddings
import vertexai
from vertexai.preview import reasoning_engines

from vertexai.preview.reasoning_engines import ReasoningEngine, LangchainAgent


### Authenticate to Google Cloud

Authenticate to Google Cloud as the IAM user logged into this notebook in order to access your Google Cloud Project.

In [4]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

### Define project information

Initialize `gcloud` with your Project ID and resource location. At this time, only `us-central1` is supported.

In [5]:
PROJECT_ID = "YOUR_PROJECT"  # @param {type:"string"}
LOCATION = "us-central1"

!gcloud config set project {PROJECT_ID}

Updated property [core/project].


## Create a Cloud Storage bucket

Create or reuse and existing Cloud Storage bucket. Reasoning engine stages the artifacts of your applications in a Cloud Storage bucket as part of the deployment process.

In [6]:
STAGING_BUCKET_NAME = "img_gemini_test"  # @param {type:"string"}
STAGING_BUCKET = f"gs://{STAGING_BUCKET_NAME}"

# Create a Cloud Storage bucket, if it doesn't already exist
!gsutil mb -c standard {STAGING_BUCKET}

Creating gs://img_gemini_test/...
ServiceException: 409 A Cloud Storage bucket named 'img_gemini_test' already exists. Try another name. Bucket names must be globally unique across all Google Cloud projects, including those outside of your organization.


### Enable APIs

This tutorial uses the following billable components of Google Cloud, which you'll need to enable for this tutorial:

In [5]:
!gcloud services enable aiplatform.googleapis.com alloydb.googleapis.com servicenetworking.googleapis.com

Operation "operations/acat.p2-273845608377-0368c134-a3de-4de3-a66c-475940e89f07" finished successfully.


## Set up AlloyDB

Use the provided variable names or update the values to use a pre-exisiting AlloyDB cluster and instance.

In [7]:
REGION = "us-central1"  # @param {type:"string"}
CLUSTER = "vector-cluster"  # @param {type:"string"}
INSTANCE = "vector-instance"  # @param {type:"string"}
DATABASE = "postgres"  # @param {type:"string"}
TABLE_NAME = "patents_data"  # @param {type:"string"}
PASSWORD = "YOUR_DB_PASSWORD"

### Create an AlloyDB cluster and primary instance

This tutorial **assumes** you already have an AlloyDB cluster, instance, database, table, data and embeddings created. It should also have public IP and IAM authentication enabled.

If not, **follow the steps 2, 3, 4 and 5 in this codelab: first**
https://codelabs.developers.google.com/patent-search-alloydb-gemini


Make sure to have **PUBLIC IP and authentication provided**

If you do not complete the codelab mentioned above, you wouldn't be able to implement the rest of the steps.

## Define the retriever tool

Tools are interfaces that an agent, chain, or LLM can use to enable the Gemini model to interact with external systems, databases, document stores, and other APIs so that the model can get the most up-to-date information or take action with those systems.

In this example, you'll define a function that will retrieve similar documents from the vector store using semantic search.

For improved security measures, the tool wil use IAM-based authentication to authenticate to the databases instead of using the built-in user/password authentication.

In [14]:
def similarity_search(query: str) -> List[Document]:
    """Searches and returns patents.

    Args:
      query: The user query to search for related items

    Returns:
      List[Document]: A list of Documents
    """
    engine = AlloyDBEngine.from_instance(
        PROJECT_ID,
        REGION,
        CLUSTER,
        INSTANCE,
        DATABASE,
        # Uncomment to use built-in authentication instead of IAM authentication
        user="postgres",
        password=PASSWORD,
    )
    vector_store = AlloyDBVectorStore.create_sync(
        engine,
        table_name=TABLE_NAME,
        embedding_service=VertexAIEmbeddings(
            model_name="textembedding-gecko@latest", project=PROJECT_ID
        ),id_column="id",content_column="abstract",embedding_column="abstract_embeddings",
    )
    retriever = vector_store.as_retriever()
    return retriever.invoke(query)

In [62]:
vector_store.similarity_search("patent related to natural language processing")

[Document(page_content='Systems and methods for identifying word collocations in natural language texts. An example method comprises: performing, by a computing device, semantico-syntactic analysis of a natural language text to produce a plurality of semantic structures; generating, in view of relationships defined by the semantic structures, a raw list of word combinations; producing a list of collocations by applying a heuristic filter to the raw list of word combinations; and using the list of collocations to perform a natural language processing operation.'),
 Document(page_content='Aspects of the present invention provide a more universal, easy, natural, and vendor-agnostic interface to configure, manage, and/or monitor devices in networks. In embodiments, a user-friendly natural language interface voice interface may be used to â€œlive chatâ€\x9d with one or more devices. In embodiments, a natural language input from a user intended for a target device is received and converted i

## Deploy the service

Now that you've specified a model, tools, and reasoning for your agent and tested it out, you're ready to deploy your agent as a remote service in Vertex AI!

Here, you'll use the LangChain agent template provided in the Vertex AI SDK for Reasoning Engine, which brings together the model, tools, and reasoning that you've built up so far.

In [16]:
vertexai.init(project=PROJECT_ID, location="us-central1", staging_bucket=STAGING_BUCKET)

remote_app = reasoning_engines.ReasoningEngine.create(
    reasoning_engines.LangchainAgent(
        model="gemini-pro",
        tools=[similarity_search],
        model_kwargs={
            "temperature": 0.1,
        },
    ),
    requirements=[
        "google-cloud-aiplatform[reasoningengine,langchain]==1.57.0",
        "langchain-google-alloydb-pg==0.4.1",
        "langchain-google-vertexai==1.0.4",
    ],
    display_name="PrebuiltAgent",
)

INFO:vertexai.reasoning_engines._reasoning_engines:Using bucket img_gemini_test
INFO:vertexai.reasoning_engines._reasoning_engines:Writing to gs://img_gemini_test/reasoning_engine/reasoning_engine.pkl
INFO:vertexai.reasoning_engines._reasoning_engines:Writing to gs://img_gemini_test/reasoning_engine/requirements.txt
INFO:vertexai.reasoning_engines._reasoning_engines:Creating in-memory tarfile of extra_packages
INFO:vertexai.reasoning_engines._reasoning_engines:Writing to gs://img_gemini_test/reasoning_engine/dependencies.tar.gz
INFO:vertexai.reasoning_engines._reasoning_engines:Creating ReasoningEngine
INFO:vertexai.reasoning_engines._reasoning_engines:Create ReasoningEngine backing LRO: projects/273845608377/locations/us-central1/reasoningEngines/6234406851350364160/operations/5119270235630731264
INFO:vertexai.reasoning_engines._reasoning_engines:ReasoningEngine created. Resource name: projects/273845608377/locations/us-central1/reasoningEngines/6234406851350364160
INFO:vertexai.reaso

## Try it out

Query the remote app directly or retrieve the application endpoint via the resource ID or display name. The endpoint can be used from any Python environment.

In [19]:
response = remote_app.query(input="Patents about natural language processing")
print(response["output"])

## Patents about natural language processing

Here are some patents related to natural language processing (NLP):

* **Systems and methods for identifying word collocations in natural language texts.** This patent describes a system and method for identifying word collocations in natural language texts. The system uses a semantic-syntactic analysis to identify relationships between words and then uses a heuristic filter to identify collocations.
* **Aspects of the present invention provide a more universal, easy, natural, and vendor-agnostic interface to configure, manage, and/or monitor devices in networks.** This patent describes a system and method for providing a natural language interface for configuring, managing, and monitoring devices in networks. The system uses a voice interface to allow users to interact with the devices in a natural way.
* **Methods and systems are provided for contextual language understanding.** This patent describes a system and method for understanding 

In [18]:
# Retrieve the application endpoint via the display name
app_list = reasoning_engines.ReasoningEngine.list(filter='display_name="PrebuiltAgent"')
RESOURCE_ID = app_list[0].name

# Retrieve the application endpoint via the resource ID
remote_app = reasoning_engines.ReasoningEngine(
    f"projects/{PROJECT_ID}/locations/{LOCATION}/reasoningEngines/{RESOURCE_ID}"
)

## Clean up

If you created a new project for this tutorial, delete the project. If you used an existing project and wish to keep it without the changes added in this tutorial, delete resources created for the tutorial.

### Deleting the project

The easiest way to eliminate billing is to delete the project that you created for the tutorial.

1. In the Google Cloud console, go to the [Manage resources](https://console.cloud.google.com/iam-admin/projects?_ga=2.235586881.1783688455.1719351858-1945987529.1719351858) page.
1. In the project list, select the project that you want to delete, and then click Delete.
1. In the dialog, type the project ID, and then click Shut down to delete the project.


### Deleting tutorial resources

Delete the reasoning engine instance(s) and AlloyDB cluster and instance.

In [ ]:
# Delete the ReasoningEngine instance
remote_app.delete()

In [ ]:
# Or delete all Reasoning Engine apps
apps = reasoning_engines.ReasoningEngine.list()
for app in apps:
    app.delete()

In [ ]:
# Delete the AlloyDB cluster and instance
!gcloud alloydb clusters delete {CLUSTER} \
  --region={REGION} \
  --project={PROJECT_ID} \
  --force \
  --quiet

## What's next

* Dive deeper into [LangChain on Vertex AI](https://cloud.google.com/vertex-ai/generative-ai/docs/reasoning-engine/overview).
* Learn more about the [AlloyDB for LangChain library](https://github.com/googleapis/langchain-google-alloydb-pg-python).
* Explore other [Reasoning Engine samples](https://github.com/GoogleCloudPlatform/generative-ai/tree/main/gemini/reasoning-engine).